## Scraping skatingscores.com

The purpose of this notebook is to test and then sucesfully scrape https://skatingscores.com/. Practically, I could ask for the data, but because this is a learning project we will scrape assuming we couldn't get it.

In [1]:
#pip install lxml html5lib beautifulsoup4
import pandas as pd
import numpy as np


Top 50 Jumps in Senior Women International
- https://skatingscores.com/q/element/?show_ranks=on&underline=&season_codes=2022&division_codes=sr&event_codes=intl&discipline_codes=women&segment_type_codes=fs&unit_country_codes=all&unit_name=%25&element_type_codes=jump&element_code=%25&sort=score&limit=50&submit=Submit

#### Element Query

In [33]:
query_df, elements_df = pd.read_html('https://skatingscores.com/q/element/?show_ranks=on&underline=&season_codes=2022&division_codes=sr&event_codes=intl&discipline_codes=women&segment_type_codes=fs&unit_country_codes=all&unit_name=%25&element_type_codes=jump&element_code=%25&sort=score&limit=50&submit=Submit')
elements_df

,Unnamed: 0,#,Nat,Skater,Jump,TotalScore,BaseValue,GOETotal,MeanGOE Mark,+5s(or +3s),Season,Event,Seg
0,1,1,🇷🇺,Kamila Valieva,4T+1Eu+3S x,19.06,15.73,3.33,3.38,0 of 8,21/22,FINTRO,FS
1,2,2,🇷🇺,Kamila Valieva,4T+1Eu+3S x,18.72,15.73,2.99,3.11,0 of 9,21/22,GPCAN,FS
2,3,3,🇷🇺,Kamila Valieva,4T+3T,17.91,13.70,4.21,4.33,4 of 9,21/22,GPRUS,FS
3,4,4,🇷🇺,Kamila Valieva,4T+3T,17.50,13.70,3.80,3.88,1 of 8,21/22,FINTRO,FS
4,5,5,🇷🇺,Kamila Valieva,4T+3T,16.82,13.70,3.12,3.33,1 of 9,21/22,GPCAN,FS
5,6,6,🇷🇺,Maiia Khromykh,4T+3T,16.55,13.70,2.85,3.00,0 of 7,21/22,BUDTRO,FS
6,7,7,🇷🇺,Kamila Valieva,4T+1Eu+2S x,14.47,12.43,2.04,2.00,0 of 9,21/22,GPRUS,FS
7,8,8,🇷🇺,Anna Shcherbakova,3Lz+3Lo x,13.99,11.88,2.11,3.56,2 of 9,21/22,GPITA,FS
8,9,8,🇷🇺,Anna Shcherbakova,4F,13.99,11.00,2.99,2.78,0 of 9,21/22,GPITA,FS
9,10,10,🇷🇺,Alexandra Trusova,4Lz,13.80,11.50,2.30,2.11,0 of 9,21/22,GPUSA,FS


Better data of what I want
- each skater can be described by a set of 11 tables


https://skatingscores.com/2021/wc/sr/women/long/

Called the "Enhanced Protocol" according to skatingscores

In [31]:
dfs = pd.read_html('https://skatingscores.com/2021/wc/sr/women/long/')
len(dfs)

265

In [128]:
def clean_technical_score(data):
    """
    scores is the 3rd table
    todo: need to add skater name
    """

    df = data.dropna(axis = 1)
    df.columns = df.iloc[0,:] #use 1st row as column
    df = df.iloc[1:-1, :] # remove last row which contains column sums and the first row containing columns names

    elements = df['Element']
    df.drop(columns  = 'Element', inplace = True)

    for i in df.columns:
        df[i] = df[i].astype(float)

    df.insert(0, 'Element', elements)

    return df
technical_score = clean_technical_score(data = dfs[3])
technical_score

,Element,BaseValue,GOETotal,HKG 🇭🇰,CZE 🇨🇿,SWE 🇸🇪,RUS 🇷🇺,SLO 🇸🇮,SVK 🇸🇰,DEN 🇩🇰,USA 🇺🇸,NED 🇳🇱
1,4F!,11.00,1.57,3.0,1.0,2.0,1.0,2.0,1.0,1.0,1.0,2.0
2,4S,9.70,-4.71,-5.0,-5.0,-5.0,-5.0,-5.0,-5.0,-4.0,-5.0,-4.0
3,4Lz+3T,15.70,3.78,4.0,3.0,3.0,4.0,4.0,3.0,3.0,3.0,3.0
4,2A,3.30,0.71,2.0,2.0,2.0,3.0,3.0,2.0,2.0,2.0,2.0
5,CCoSp4,3.50,0.55,1.0,2.0,0.0,1.0,2.0,1.0,2.0,2.0,2.0
6,ChSq111,3.00,1.14,2.0,3.0,2.0,2.0,3.0,2.0,3.0,2.0,1.0
7,4Lz<,10.12,-4.60,-5.0,-5.0,-5.0,-5.0,-5.0,-5.0,-5.0,-5.0,-5.0
8,4T<<+1Eu+3Sq,9.90,-1.97,-4.0,-5.0,-5.0,-4.0,-3.0,-5.0,-4.0,-5.0,-5.0
9,3Lz+3T,11.11,1.52,3.0,3.0,2.0,3.0,3.0,2.0,1.0,2.0,3.0
10,FCSp4,3.20,0.73,2.0,3.0,2.0,3.0,2.0,2.0,2.0,2.0,3.0


In [129]:

def clean_component_score(data):
    df = data
    df.columns = data.iloc[0,:]
    df = df.loc[:,~df.columns.duplicated()]
    df = df.iloc[1:-1, :-1]
    return df

component_score = clean_component_score(data = dfs[10])
component_score

,Component,Factor,HKG 🇭🇰,CZE 🇨🇿,SWE 🇸🇪,RUS 🇷🇺,SLO 🇸🇮,SVK 🇸🇰,DEN 🇩🇰,USA 🇺🇸,NED 🇳🇱
1,Skating Skills,1.60,2 9.00,3 8.75,9 8.25,2 8.75,8 8.25,6 8.50,1 9.25,3 8.75,6 8.50
2,Transitions,1.60,2 8.75,6 8.25,11 7.50,6 8.00,11 7.00,9 8.00,2 8.75,11 7.75,12 7.50
3,Performance,1.60,3 8.50,8 8.00,11 7.50,5 8.25,6 8.25,6 8.50,3 8.75,10 7.75,10 8.00
4,Composition,1.60,2 8.75,7 8.50,12 7.75,3 8.50,6 8.25,6 8.50,2 8.75,11 8.00,5 8.50
5,Interpretation,1.60,2 8.50,9 8.25,12 7.50,3 8.50,9 8.00,6 8.50,1 9.00,12 7.75,6 8.25


In [135]:

def get_skater_bio(data):
    name = data[1][0]
    country = data[2][0]
    return name, country

get_skater_bio(dfs[1])


('Alexandra TRUSOVA', 'RUS')

In [ ]:
class scrapePrograms():


    

In [134]:
for i in np.arange(11):
    print(i)
    display(dfs[i].head(), dfs[i].shape)

0


,0,1,2,3
0,« Prior Season,SP,FS,NaN


(1, 4)

1


,0,1,2,3,4,5
0,1,Alexandra TRUSOVA,RUS,Start,Ded,Total
1,1,Alexandra TRUSOVA,RUS,13,-2.00,152.38


...

,0,1
0,Technical Element Score,1 88.04


(1, 2)

10


,Component,Component,Component,Factor,Factor,Factor,HKG 🇭🇰,CZE 🇨🇿,SWE 🇸🇪,RUS 🇷🇺,SLO 🇸🇮,SVK 🇸🇰,DEN 🇩🇰,USA 🇺🇸,NED 🇳🇱,Score
0,Component,Component,Component,Factor,Factor,Factor,HKG 🇭🇰,CZE 🇨🇿,SWE 🇸🇪,RUS 🇷🇺,SLO 🇸🇮,SVK 🇸🇰,DEN 🇩🇰,USA 🇺🇸,NED 🇳🇱,Score
1,Skating Skills,Skating Skills,Skating Skills,1.60,1.60,1.60,2 9.00,3 8.75,9 8.25,2 8.75,8 8.25,6 8.50,1 9.25,3 8.75,6 8.50,58.64
2,Transitions,Transitions,Transitions,1.60,1.60,1.60,2 8.75,6 8.25,11 7.50,6 8.00,11 7.00,9 8.00,2 8.75,11 7.75,12 7.50,97.96
3,Performance,Performance,Performance,1.60,1.60,1.60,3 8.50,8 8.00,11 7.50,5 8.25,6 8.25,6 8.50,3 8.75,10 7.75,10 8.00,88.18
4,Composition,Composition,Composition,1.60,1.60,1.60,2 8.75,7 8.50,12 7.75,3 8.50,6 8.25,6 8.50,2 8.75,11 8.00,5 8.50,78.43


(7, 16)